In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
frames = c_armor.time.size

#%matplotlib notebook
fig, ax = plt.subplots(nrows=1, ncols=1, subplot_kw=dict(projection=projection), 
                        figsize=(15, 7.5))

ax.set_global()
ax.coastlines('50m')
ax.set_xticks([-90, -75, -60, -45, -30, -15, 0], crs=ccrs.PlateCarree())
ax.set_yticks([10, 20, 30, 40, 50, 60, 70], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()

ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
ax.set_extent(region, crs=ccrs.PlateCarree())     #lat, lon
ax.gridlines()


def draw(frame):
    x_n, y_n = Grid(var2)
    vmin = Range_MLD[month]['vmin']
    vmax = Range_MLD[month]['vmax']
    
    contr1 = ax.contour(x_n, y_n, var1, 10, cmap='RdYlBu')
    contr2 = ax.contourf(x_n, y_n, var2, cmap='bone_r')
    ax.set_title('')

    fig.colorbar(contr1, ax=ax, shrink=0.55, orientation='vertical', label='GH [m]')
    fig.colorbar(contr2, ax=ax, shrink=0.55, orientation='horizontal', label='MLD [m]')
    plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9, wspace = 0.3)
    return contr1


def init():
    return draw(0)


def animate(frame):
    return draw(frame)

ani = animation.FuncAnimation(fig, animate, init_func=init, frames=frames, 
                              interval=0.01, blit=False, repeat=False)
#plt.show()
